# **Convolutional Neural Network**
*   Implementation of a convolutional neural network classifier supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a convolutional neural network classifier supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementationa are based off of TensorFlow's very own example: [TensorFlow CNN Tutorial](https://www.tensorflow.org/tutorials/images/cnn)

In [1]:
import time
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [20]:
# Set a global random seed
tf.random.set_seed(12)

In [3]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 2
# Values are specific to the CIFAR10 dataset
INPUT_DIM = 32
INPUT_CHANNELS = 3
NUM_CLASSES = 10

In [4]:
def build_and_train(X_train, y_train, precision='single'):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS), dtype=dtype),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', dtype=dtype),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(dtype=dtype),
        layers.Dense(64, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, activation='softmax', dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [5]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [17]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test = X_test / 255.0

In [7]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1563/1563 [==============================] - 12s 5ms/step - loss: 1.4997 - accuracy: 0.4565
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1528 - accuracy: 0.5935
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.0044 - accuracy: 0.6490
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9034 - accuracy: 0.6831
Epoch 5/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8359 - accuracy: 0.7071
Epoch 1/5
1563/1563 [==============================] - 14s 6ms/step - loss: 1.4793 - accuracy: 0.4648
Epoch 2/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1216 - accuracy: 0.6042
Epoch 3/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9678 - accuracy: 0.6610
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8767 - accuracy: 0.6930
Epoch 5/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8057 - accuracy: 0.717

In [8]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 18s 9ms/step - loss: 1.5288 - accuracy: 0.4404
Epoch 2/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.1716 - accuracy: 0.5880
Epoch 3/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.0246 - accuracy: 0.6402
Epoch 4/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.9260 - accuracy: 0.6776
Epoch 5/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.8649 - accuracy: 0.6978
Epoch 1/5
1563/1563 [==============================] - 16s 8ms/step - loss: 1.5126 - accuracy: 0.4468
Epoch 2/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.1785 - accuracy: 0.5810
Epoch 3/5
1563/1563 [==============================] - 13s 8ms/step - loss: 1.0207 - accuracy: 0.6411
Epoch 4/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.9181 - accuracy: 0.6797
Epoch 5/5
1563/1563 [==============================] - 13s 8ms/step - loss: 0.8485

In [9]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.5431 - accuracy: 0.4371
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1720 - accuracy: 0.5858
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.0090 - accuracy: 0.6458
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9152 - accuracy: 0.6802
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: 0.8445 - accuracy: 0.7060
Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: 1.5262 - accuracy: 0.4471
Epoch 2/5
1563/1563 [==============================] - 7s 4ms/step - loss: 1.1783 - accuracy: 0.5818
Epoch 3/5
1563/1563 [==============================] - 7s 5ms/step - loss: 1.0192 - accuracy: 0.6404
Epoch 4/5
1563/1563 [==============================] - 7s 4ms/step - loss: 0.9194 - accuracy: 0.6775
Epoch 5/5
1563/1563 [==============================] - 7s 5ms/step - loss: 0.8501 - accurac

In [21]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 2/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 7s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: nan - accuracy: 0.1001
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1563/1563 [==============================] - 8s 5ms/step - loss: nan - accuracy: 0.1000
313/313 - 1s - loss:

In [23]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1563/1563 [==============================] - 11s 6ms/step - loss: 1.5121 - accuracy: 0.4515
Epoch 2/5
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1664 - accuracy: 0.5887
Epoch 3/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.0071 - accuracy: 0.6479
Epoch 4/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9078 - accuracy: 0.6834
Epoch 5/5
1563/1563 [==============================] - 9s 5ms/step - loss: 0.8389 - accuracy: 0.7043
Epoch 1/5
1563/1563 [==============================] - 11s 6ms/step - loss: 1.5238 - accuracy: 0.4451
Epoch 2/5
1563/1563 [==============================] - 9s 6ms/step - loss: 1.1568 - accuracy: 0.5904
Epoch 3/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.9964 - accuracy: 0.6492
Epoch 4/5
1563/1563 [==============================] - 9s 6ms/step - loss: 0.8997 - accuracy: 0.6848
Epoch 5/5
1563/1563 [==============================] - 9s 5ms/step - loss: 0.8341 - accur

In [24]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 78.65223252773285 seconds
Average training time in single precision: 39.74348986148834 seconds
Average training time in half precision: 42.74839901924133 seconds
Average training time in mixed half precision: 48.327919721603394 seconds
-------------
Accuracy with double precision: 0.6747999787330627
Accuracy with single precision: 0.6773999929428101
Accuracy with half precision: 0.10000000149011612
Accuracy with mixed half precision: 0.6834999918937683
